In [85]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

import csv
import json
import time
import urllib
import requests
import pandas as pd

In [86]:
df = pd.read_csv("movie_ids.csv", converters={'movie_ids': lambda x: str(x)})
ids = df["movie_ids"]
ids = list(df.iloc[:5]["movie_ids"])
len(ids)

5

In [87]:
req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"}) 

chrome_options = webdriver.ChromeOptions(); 
chrome_options.add_experimental_option("excludeSwitches", ['enable-automation']);

In [88]:
reviews = []
ratings = []

In [89]:
def extract_ratings():
    rating_reviews = driver.find_elements_by_xpath(".//div[@class='review-container']")
    for rating_review in rating_reviews:
        try:
            new_rating = rating_review.find_element_by_xpath(".//span[@class='rating-other-user-rating']/span[1]")
            ratings.append(new_rating.text)
        except NoSuchElementException:  
            ratings.append(None)

In [90]:
def extract_reviews():
    rating_reviews = driver.find_elements_by_xpath(".//div[@class='review-container']")
    for rating_review in rating_reviews:
        review = rating_review.find_element_by_xpath(".//div[@class='content']/div[@class='text show-more__control']")
        reviews.append(review.text)

In [91]:
sum = 0
for id in ids: 
    sum += 1
    url = "https://www.imdb.com/title/tt" + id + "/reviews"
    driver = webdriver.Chrome(options=chrome_options)    

    driver.get(url)

    show_more_button = driver.find_elements_by_class_name("ipl-load-more__button")
    
    if show_more_button != []:
        while show_more_button[0].value_of_css_property("display") == "block":
            show_more_button[0].click() 
            time.sleep(3)

    read_full_buttons = driver.find_elements_by_xpath(".//div[@class='gradient-expander  show-more']")

    # Click on read more button to read
    non_spoiler =[]

    #  Non spoiler review
    for read_full_button in read_full_buttons:
        full_review = read_full_button.find_element_by_class_name("expander-icon-wrapper")

        non_spoiler.append(full_review)

    # Spoiler review
    spoiler = driver.find_elements_by_xpath(".//div[@class='expander-icon-wrapper spoiler-warning__control']")

    show_more_reviews = non_spoiler + spoiler

    for show_more_review in show_more_reviews:
        show_more_review.click()


    if show_more_button != []:
        if show_more_button[0].value_of_css_property("display") == "none":
            extract_ratings()
            extract_reviews()
    
    print(sum)

    driver.close()


1


WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=87.0.4280.66)


In [ ]:
dataframe = pd.DataFrame({"reviews": reviews, "ratings": ratings})

dataframe.to_csv('movie_review.csv')

In [ ]:
print(dataframe.shape)
dataframe.head()